In [1]:
import pandas as pd
from sklearn.cluster import KMeans
from patternly.detection import AnomalyDetection

data = pd.read_csv("./data1", low_memory=False)
seqs = pd.concat(
    [data[(1000 * i):(1000 * (i + 1))].reset_index(drop=True) for i in range(int(data.shape[0]/1000))], 
    axis=1
)
seqs = seqs.T
seqs.index = range(seqs.shape[0])
seqs

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0,1,0,0,1,0,1,0,1,1,...,1,0,1,0,1,0,1,1,0,1
1,0,1,0,1,0,1,0,1,0,0,...,1,0,1,0,1,0,0,1,0,1
2,0,1,0,1,0,1,1,0,1,0,...,0,1,0,1,0,1,0,1,0,1
3,1,0,1,0,1,0,1,1,0,1,...,0,1,0,1,0,1,1,0,1,1
4,0,1,0,0,1,0,1,1,0,1,...,0,1,0,1,0,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,0,1,0,1,0,1,0,0,1,0,...,1,0,0,1,0,1,0,1,0,1
236,0,1,0,0,1,0,1,0,0,1,...,0,1,0,1,0,1,0,1,1,0
237,1,0,1,0,1,0,1,1,0,1,...,0,1,1,0,0,1,0,1,1,0
238,0,1,0,0,0,1,0,1,0,0,...,0,1,0,1,0,1,0,0,1,0


In [2]:
kmeans = KMeans(n_clusters=1)
pipeline = AnomalyDetection(anomaly_sensitivity=2, clustering_alg=kmeans, quantize=False, eps=0.1, verbose=True)
pipeline.fit(seqs)

predictions = pd.DataFrame(pipeline.predict())
anomalies = predictions[predictions[0] == True]
anomalies
print(list(anomalies.index))

Calculating distance matrix...
Clustering distance matrix...
Generating cluster PFSA 1/1...
Calculating cluster PFSA means and stds...
[27, 96, 200, 209, 239]


In [5]:
pipeline.predict(seqs.iloc[239], clean=False)

True

In [4]:
pipeline.curr_cluster_llk_vec

array([[0.73034211]])